In [1]:
!ls

app.py                data_import.py        requirements.txt
chroma.ipynb          generate_relations.py services
comments.txt          logo-femto.png
create_assistant.py   nl2cypher


In [10]:
import numpy as np 
import chromadb
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
client = AzureOpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            api_version=os.getenv("OPENAI_API_VERSION"),
            azure_endpoint=os.getenv("OPENAI_API_BASE"),
        )

In [13]:
import os
import json
import re

# Define the path to the tables
relations_path = "nl2cypher/relations"

relations = []
for filename in os.listdir(relations_path):
    with open(os.path.join(relations_path, filename)) as f:
        relations_list = json.load(f)
        for relation in relations_list:
            relations.append({
                "relation": relation["relation"],
                "description": relation["description"]
        })

relations

[{'relation': 'Capteur01,IS_INSTALLED_IN,Mairie,"{installation_date: \'2023-06-15\', last_maintenance: \'2023-12-10\'}"',
  'description': 'Capteur01 is a Thing that has a relationship IS_INSTALLED_IN with Mairie, which is a Location. This indicates the physical location where the sensor or device is installed. The relationship includes metadata like installation and maintenance info.'},
 {'relation': 'Capteur02,IS_INSTALLED_IN,Ecole_Primaire,"{installation_date: \'2023-05-20\', last_maintenance: \'2023-11-05\'}"',
  'description': 'Capteur02 is a Thing that has a relationship IS_INSTALLED_IN with Ecole_Primaire, which is a Location. This indicates the physical location where the sensor or device is installed. The relationship includes metadata like installation and maintenance info.'},
 {'relation': 'Capteur03,IS_INSTALLED_IN,Ecole_Maternelle,"{installation_date: \'2023-05-20\', last_maintenance: \'2023-11-05\'}"',
  'description': 'Capteur03 is a Thing that has a relationship IS_INST

In [15]:
embeddings = [client.embeddings.create(input=relation['description'],model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")).data[0].embedding for relation in relations] 
print(len(embeddings))
embeddings[0]

228


[0.0016665008151903749,
 0.00857346411794424,
 0.0012519819429144263,
 -0.0031425924971699715,
 -0.012368165887892246,
 0.026192203164100647,
 -0.003851992776617408,
 -0.0014339657500386238,
 0.004488936625421047,
 -0.01845451630651951,
 -0.005836965516209602,
 0.012705173343420029,
 -0.024156678467988968,
 -0.01792878471314907,
 0.004836054053157568,
 -0.010143917985260487,
 0.014032981358468533,
 -0.026974059641361237,
 -0.004761912394315004,
 -0.016378551721572876,
 0.0034290486946702003,
 -0.005173061043024063,
 -0.030438493937253952,
 -0.002719648415222764,
 -0.011370624415576458,
 -0.005317974369972944,
 0.011572828516364098,
 -0.027715476229786873,
 -0.012374905869364738,
 0.0013328635832294822,
 0.035803649574518204,
 -0.024547606706619263,
 -0.04248987138271332,
 -0.03324239328503609,
 -0.019694702699780464,
 -0.0131163215264678,
 0.01660771667957306,
 -0.011963756754994392,
 -0.0013126431731507182,
 -0.01493616122752428,
 0.026273084804415703,
 -0.0015375955263152719,
 0.0004

In [16]:
# Add the project root directory to sys.path
import sys
project_root = os.path.abspath(os.path.join(os.getcwd()))
print(project_root)
sys.path.append(project_root)

/Users/josephazar/Desktop/MY_WORK/proof_of_concepts/cypher2sql-poc/backend


In [17]:
# Initialize ChromaDB Client
chroma_client = chromadb.PersistentClient(path=os.path.join(project_root, "chromadb"))

In [18]:
chroma_client.list_collections()

[]

In [19]:
collections = chroma_client.list_collections()

# Delete each collection
for collection in collections:
    chroma_client.delete_collection(collection)
    print(f"Deleted collection: {collection}")

# Verify collections are deleted
print("Remaining collections:", chroma_client.list_collections())

Remaining collections: []


In [20]:
# Create a collection in ChromaDB to store embeddings
collection_name = "nl2cypher-relations"
collection = chroma_client.create_collection(name=collection_name)

In [21]:
# add relations to collection
for relation, embedding in zip(relations, embeddings):
    try:
        # Attempt to add the album to the collection
        collection.add(
            embeddings=[embedding],  # Ensure embedding is a list of floats
            documents=[relation['description']],
            metadatas=[{'relation': relation['relation']}],
            ids=[relation['relation'].replace(' ', '_').lower()]
        )
        # Print the success message after adding the album
        print(f"Successfully added relation {relation['relation']}")
    except Exception as e:
        # Print the error message if the addition fails
        print(f"Error adding relation {relation['relation']}: {e}")

Successfully added relation Capteur01,IS_INSTALLED_IN,Mairie,"{installation_date: '2023-06-15', last_maintenance: '2023-12-10'}"
Successfully added relation Capteur02,IS_INSTALLED_IN,Ecole_Primaire,"{installation_date: '2023-05-20', last_maintenance: '2023-11-05'}"
Successfully added relation Capteur03,IS_INSTALLED_IN,Ecole_Maternelle,"{installation_date: '2023-05-20', last_maintenance: '2023-11-05'}"
Successfully added relation Capteur04,IS_INSTALLED_IN,Ecole_Maternelle,"{installation_date: '2023-05-20', last_maintenance: '2023-11-05'}"
Successfully added relation Capteur05,IS_INSTALLED_IN,Salle_des_Fetes,"{installation_date: '2023-07-30', last_maintenance: '2024-01-15'}"
Successfully added relation Capteur06,IS_INSTALLED_IN,Place_de_la_Poste,"{installation_date: '2023-08-12', last_maintenance: '2024-02-05'}"
Successfully added relation Capteur07,IS_INSTALLED_IN,Smart_Grid,"{installation_date: '2023-09-25', last_maintenance: '2024-03-10'}"
Successfully added relation Capteur08,IS_INST

In [22]:
query_embedding =  client.embeddings.create(input="Tous les capteurs installés à la mairie",model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")).data[0].embedding

results = collection.query(
    #query_texts=["Top 5 customers with highest balance outstanding"],
    query_embeddings=[query_embedding],
    n_results=10,
    include=["embeddings", "documents", "metadatas", "distances"]
    )
results

{'ids': [['capteur01,is_installed_in,mairie,"{installation_date:_\'2023-06-15\',_last_maintenance:_\'2023-12-10\'}"',
   'capteur20,is_installed_in,mairie,"{installation_date:_\'2023-06-15\',_last_maintenance:_\'2023-12-10\'}"',
   'capteur13,is_installed_in,mairie,"{installation_date:_\'2023-07-15\',_last_maintenance:_\'2024-01-20\'}"',
   'capteur17,is_installed_in,toiture_mairie,"{installation_date:_\'2023-06-18\',_last_maintenance:_\'2023-12-20\'}"',
   'capteur08,is_installed_in,toiture_mairie,"{installation_date:_\'2023-06-18\',_last_maintenance:_\'2023-12-20\'}"',
   'capteur18,is_installed_in,toiture_mairie,"{installation_date:_\'2023-06-18\',_last_maintenance:_\'2023-12-20\'}"',
   'capteur14,is_installed_in,mairie,"{installation_date:_\'2023-10-10\',_last_maintenance:_\'2024-02-25\'}"',
   'capteur13,is_installed_in,ateliers_municipaux,"{installation_date:_\'2023-07-15\',_last_maintenance:_\'2024-01-20\'}"',
   'capteur05,is_installed_in,salle_des_fetes,"{installation_date:_\

In [23]:
# Define the path to the tables
queries_path = "nl2cypher/queries"

queries = []
for filename in os.listdir(queries_path):
    with open(os.path.join(queries_path, filename)) as f:
        queries_list = json.load(f)
        for query in queries_list:
            queries.append({
                "question": query["question"],
                "cypher": query["cypher"]
        })

queries

[{'question': "Quels sont les capteurs installés à l'École Maternelle ?",
  'cypher': "MATCH (t:Thing)-[:IS_INSTALLED_IN]->(l:Location {name: 'École Maternelle'}) RETURN t.identifier AS capteur_id, t.name AS nom_capteur"},
 {'question': 'Quels capteurs mesurent la température ?',
  'cypher': "MATCH (t:Thing)-[:IS_COMPONENT_OF]->(s:Sensor {name: 'Capteur Température'}) RETURN t.identifier AS capteur_id, t.name AS nom_capteur"},
 {'question': 'Quel est le dernier emplacement de Capteur07 ?',
  'cypher': "MATCH (t:Thing {identifier: 'Capteur07'})-[:IS_INSTALLED_IN]->(l:Location) RETURN l.name AS emplacement"},
 {'question': 'Quels capteurs sont alimentés par panneaux solaires ?',
  'cypher': "MATCH (t:Thing)-[:IS_POWERED_BY]->(p:Power {name: 'Panneaux Solaires'}) RETURN t.identifier AS capteur_id, t.name AS nom_capteur"},
 {'question': 'Quels capteurs utilisent le réseau LoRaWAN ?',
  'cypher': "MATCH (t:Thing)-[:IS_USING]->(n:Network {name: 'Réseau LoRaWAN'}) RETURN t.identifier AS capte

In [24]:
# Create a collection in ChromaDB to store queries
collection_name = "nl2cypher-queries"
collection = chroma_client.create_collection(name=collection_name)

In [25]:
chroma_client.list_collections()

[Collection(name=nl2cypher-relations), Collection(name=nl2cypher-queries)]

In [26]:
# Generate embeddings for queries questions
embeddings =  [client.embeddings.create(input=query['question'],model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")).data[0].embedding for query in queries]
embeddings[0]

[0.00697275111451745,
 0.013489103876054287,
 -0.0108077647164464,
 -0.01697547920048237,
 -0.033570628613233566,
 0.027358541265130043,
 0.008322929963469505,
 -0.005641589872539043,
 -0.008367301896214485,
 0.00636738957837224,
 0.02219870500266552,
 0.008158119395375252,
 -0.03595403954386711,
 -0.007061495445668697,
 0.0035370865371078253,
 -0.010345027782022953,
 0.023415766656398773,
 0.0010918694315478206,
 -0.0005360302748158574,
 -0.01075071468949318,
 0.0032375752925872803,
 0.00573984207585454,
 -0.0073530832305550575,
 0.003752608085051179,
 -0.008690583519637585,
 -0.01904194988310337,
 0.01711493544280529,
 -0.018091119825839996,
 0.00569547014310956,
 -0.007124884054064751,
 0.03476233407855034,
 -0.008988509885966778,
 -0.02377074398100376,
 -0.039478451013565063,
 -0.04186186194419861,
 0.0010356120765209198,
 0.01702619157731533,
 0.017368489876389503,
 0.019663158804178238,
 0.0241257194429636,
 0.02103235386312008,
 0.013501781970262527,
 -0.002594180405139923,
 -0.

In [27]:
for query, embedding in zip(queries, embeddings):
    try:
        # Attempt to add the query to the collection
        collection.add(
            embeddings=[embedding],  # Ensure embedding is a list of floats
            documents=[query['question']],
            metadatas=[{'query': query['cypher']}],
            ids=[query['question'].replace(' ', '_').lower()]
        )
        # Print the success message after adding the album
        print(f"Successfully added query {query['question']}")
    except Exception as e:
        # Print the error message if the addition fails
        print(f"Error adding query {query['question']}: {e}")

Successfully added query Quels sont les capteurs installés à l'École Maternelle ?
Successfully added query Quels capteurs mesurent la température ?
Successfully added query Quel est le dernier emplacement de Capteur07 ?
Successfully added query Quels capteurs sont alimentés par panneaux solaires ?
Successfully added query Quels capteurs utilisent le réseau LoRaWAN ?
Successfully added query Quels capteurs sont installés à Mairie ?
Successfully added query Quels capteurs sont installés à École Primaire ?
Successfully added query Quels capteurs sont installés à Salle des Fêtes ?
Successfully added query Quels capteurs sont installés à Station Hydrogene ?
Successfully added query Quels capteurs mesurent avec Capteur Température ?
Successfully added query Quels capteurs mesurent avec Capteur CO2 ?
Successfully added query Quels capteurs mesurent avec Capteur Pression ?
Successfully added query Quels capteurs utilisent le réseau Réseau LoRaWAN ?
Successfully added query Quels capteurs utili

In [28]:
query_embedding =  client.embeddings.create(input="Tous les capteurs installés à la mairie",model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")).data[0].embedding

results = collection.query(
    #query_texts=["Top 5 customers with highest balance outstanding"],
    query_embeddings=[query_embedding],
    n_results=10,
    include=["embeddings", "documents", "metadatas", "distances"]
    )
results

{'ids': [['quels_capteurs_sont_installés_à_mairie_?',
   "quels_capteurs_de_type_'station_météo'_sont_installés_à_la_toiture_de_la_mairie_?",
   "quels_capteurs_installés_à_la_mairie_envoient_leurs_données_à_plus_d'une_application_?",
   'quels_capteurs_installés_dans_la_mairie_mesurent_le_co2_et_sont_connectés_en_wifi_?',
   'quels_capteurs_installés_à_la_mairie_sont_alimentés_par_panneaux_solaires_?',
   'quels_capteurs_mesurent_la_consommation_énergétique_et_sont_installés_sur_la_toiture_de_la_mairie_?',
   'quels_capteurs_sont_installés_à_salle_des_fêtes_?',
   'quelles_sont_les_combinaisons_de_fournisseurs_et_de_fabricants_pour_les_capteurs_installés_à_la_mairie_?',
   'combien_de_capteurs_sont_installés_à_chaque_emplacement_?',
   'quels_capteurs_sont_installés_à_école_primaire_?']],
 'embeddings': [array([[ 1.27502289e-02,  8.38727970e-03,  2.93685030e-03, ...,
          -2.46962416e-03, -3.47212553e-02,  6.96118176e-03],
         [ 1.12248566e-02,  3.22440150e-03, -1.03255117e-